## Week Three Capstone Assignment part B

In [1]:
%reset -f
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from lxml import etree
import geocoder

## Condensed Notebook cells from part A and B

In [5]:
## collect data from url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
## parse url with beautiful soup
html_code = r.content
soup = bs(html_code,'html.parser')
## select table code from parsed code
table = soup.select("table")[0]
## create dataframe
df = pd.DataFrame(pd.read_html(str(table))[0])

## this function was built with the assignment instructions
def get_lat_long(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    return lat_lng_coords 

## Start Part C

### Grab all data with Toronto in the Borough name
### This will also remove any 'Not assigned' boroughs

In [11]:
## grab all rows with 'Toronto' in the Borough
df_toronto = df[df['Borough'].str.contains('Toronto')]

## reset index
df_toronto.reset_index(drop=True,inplace=True)
df_toronto

,Postal Code,Borough,Neighbourhood
0,M5A,Downtown Toronto,"Regent Park, Harbourfront"
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
2,M5B,Downtown Toronto,"Garden District, Ryerson"
3,M5C,Downtown Toronto,St. James Town
4,M4E,East Toronto,The Beaches
5,M5E,Downtown Toronto,Berczy Park
6,M5G,Downtown Toronto,Central Bay Street
7,M6G,Downtown Toronto,Christie
8,M5H,Downtown Toronto,"Richmond, Adelaide, King"
9,M6H,West Toronto,"Dufferin, Dovercourt Village"


### Check for uncombined cities

In [24]:
## first group the dataframe by postal code then neighbourhood
g = df_toronto.groupby(['Postal Code','Neighbourhood'])

## count the size of each group
g = g.count()

## print the number of groups that have more than one
count = len(g[g['Borough'] > 1])

print('Number of Neighbourhoods that need to be grouped: {} '.format(count))

Number of Neighbourhoods that need to be grouped: 0 


Add new columns to dataframe

In [29]:
## initialize new columns
df_toronto['Latitude'] = ""
df_toronto['Longitude'] = ""
df_toronto.columns

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Index(['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [31]:
## read the csv, find matching postal codes and fill in the latitude and longitude
with open('./Geospatial_Coordinates.csv','r') as f:                     # open the csv file
    for line in f.readlines():                                          # for every line in the csv file
        postalcode, lat, lon = line.split(',')
        # split the line at every , and place into postal, lat, long
        df_toronto.loc[df_toronto['Postal Code'] == postalcode,'Latitude'] = lat.strip()
        df_toronto.loc[df_toronto['Postal Code'] == postalcode,'Longitude'] = lon.strip()
        
df_toronto

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623015,-79.3894938
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6571618,-79.3789371
3,M5C,Downtown Toronto,St. James Town,43.6514939,-79.3754179
4,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
5,M5E,Downtown Toronto,Berczy Park,43.6447708,-79.3733064
6,M5G,Downtown Toronto,Central Bay Street,43.6579524,-79.3873826
7,M6G,Downtown Toronto,Christie,43.669542,-79.4225637
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6505712,-79.3845675
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6690051,-79.4422593


Final Dataframe

In [32]:
## postal code does  not match the assignment photo
df_toronto.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_toronto

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623015,-79.3894938
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6571618,-79.3789371
3,M5C,Downtown Toronto,St. James Town,43.6514939,-79.3754179
4,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
5,M5E,Downtown Toronto,Berczy Park,43.6447708,-79.3733064
6,M5G,Downtown Toronto,Central Bay Street,43.6579524,-79.3873826
7,M6G,Downtown Toronto,Christie,43.669542,-79.4225637
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6505712,-79.3845675
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6690051,-79.4422593
